# Knowledge Editing from IPC to BNS - GPT using WISE

## Cloning the library's repository

In [1]:
!git clone https://github.com/zjunlp/EasyEdit

Cloning into 'EasyEdit'...
remote: Enumerating objects: 7366, done.
remote: Counting objects: 100% (2597/2597), done.
remote: Compressing objects: 100% (426/426), done.
remote: Total 7366 (delta 2367), reused 2171 (delta 2171), pack-reused 4769 (from 4)
Receiving objects: 100% (7366/7366), 76.58 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (4876/4876), done.


In [2]:
%cd EasyEdit


/content/EasyEdit


## Installing packages

#### Manual installation since requirements.txt present in EasyEditor is incompatible with python 3.11 on collab

In [3]:
!pip install --upgrade datasets gpustat hydra-core higher omegaconf sentence-transformers openai peft timm iopath fairscale zhipuai av qwen_vl_utils datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 21.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<strong> ‼️‼️‼️ Please Restart the session after installing the packages! </strong>

In [1]:
%cd EasyEdit

/content/EasyEdit


## Importing EasyEdit

In [2]:
import torch
from easyeditor import BaseEditor
from easyeditor import WISEHyperParams #Using the WISE algorithm to knowledge edit the model


/usr/local/lib/python3.11/dist-packages/timm/models/hub.py:4: FutureWarning: Importing from timm.models.hub is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


## Importing the model

**‼️Replace the model name to gpt2-xl or gpt2-large accordingly‼️**

In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt2_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-medium").to(device)  # change the model to gpt2-large/gpt2-xl
gpt2_tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-medium")  # change the tokenizer to gpt2-large/gpt2-xl

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
gpt2_model.device #make sure its cuda :)

device(type='cuda', index=0)

### Hyperparameters file for gpt2-large

#### ‼️‼️ Hparams file for gpt2-medium ‼️‼️

In [5]:


gpt2_file="""
alg_name: "WISE"
model_name: "openai-community/gpt2-medium"
device: 0

mask_ratio: 0.2
edit_lr: 1.0
n_iter: 40
norm_constraint: 1.0
act_margin: [5.0, 20.0, 10.0] # alpha, beta, gamma
act_ratio: 0.8
save_freq: 500
merge_freq: 1000
merge_alg: 'ties'
objective_optimization: 'only_label'
inner_params:
- transformer.h[8].mlp.c_fc.weight


## alternative: WISE-Merge, WISE-Retrieve

# for merge (if merge)
densities: 0.53
weights: 1.0

# for retrieve (if retrieve, pls set to True)
retrieve: True
replay: False # True --> will replay the past editing instances: see https://arxiv.org/abs/2405.14768 Appendix B.3

"""
with open("./hparams/WISE/gpt2-medium.yaml","w") as f :
  print(f.write(gpt2_file))

610


#### ‼️‼️ for gpt2-large ‼️‼️

In [6]:


gpt2_large_file="""
alg_name: "WISE"
model_name: "openai-community/gpt2-large"
device: 0

mask_ratio: 0.2
edit_lr: 1.0
n_iter: 40
norm_constraint: 1.0
act_margin: [5.0, 20.0, 10.0] # alpha, beta, gamma
act_ratio: 0.8
save_freq: 500
merge_freq: 1000
merge_alg: 'ties'
objective_optimization: 'only_label'
inner_params:
- transformer.h[8].mlp.c_fc.weight


## alternative: WISE-Merge, WISE-Retrieve

# for merge (if merge)
densities: 0.53
weights: 1.0

# for retrieve (if retrieve, pls set to True)
retrieve: True
replay: False # True --> will replay the past editing instances: see https://arxiv.org/abs/2405.14768 Appendix B.3

"""
with open("./hparams/WISE/gpt2-large.yaml","w") as f :
  print(f.write(gpt2_large_file))

609


#### ‼️‼️ for gpt2-xl ‼️‼️

In [7]:


gpt2_xl_file="""
alg_name: "WISE"
model_name: "openai-community/gpt2-xl"
device: 0

mask_ratio: 0.2
edit_lr: 1.0
n_iter: 40
norm_constraint: 1.0
act_margin: [5.0, 20.0, 10.0] # alpha, beta, gamma
act_ratio: 0.8
save_freq: 500
merge_freq: 1000
merge_alg: 'ties'
objective_optimization: 'only_label'
inner_params:
- transformer.h[8].mlp.c_fc.weight


## alternative: WISE-Merge, WISE-Retrieve

# for merge (if merge)
densities: 0.53
weights: 1.0

# for retrieve (if retrieve, pls set to True)
retrieve: True
replay: False # True --> will replay the past editing instances: see https://arxiv.org/abs/2405.14768 Appendix B.3

"""
with open("./hparams/WISE/gpt2-xl.yaml","w") as f :
  print(f.write(gpt2_xl_file))

606


## Importing the datasets

### Editing and Rephrased Dataset

In [8]:
import pandas as pd
editing_data=pd.read_csv("/content/bns_ke_data.csv")
editing_prompts=editing_data["question"].to_list()
rephrased_prompts=editing_data["rephrased_prompts"].to_list()
editing_targets=editing_data["answer"].to_list()
assert len(editing_prompts)==len(editing_targets), "lengths of prompts and targets should be equal"
print('Length of rephrased prompts',len(rephrased_prompts))
print('Length of editing prompts',len(editing_prompts))
print(editing_data.loc[:5])

Length of rephrased prompts 720
Length of editing prompts 720
                                            question  \
0  What does BNS stand for in the context of Indi...   
1  When was BNS introduced in the context of Indi...   
2  What law did BNS replace in the context of Ind...   
3  Is BNS applicable across all of India in the c...   
4  What is the main purpose of BNS in the context...   
5  Does BNS cover both civil and criminal laws in...   

                                   rephrased_prompts  \
0          What is the meaning of BNS in Indian law?   
1             When was BNS introduced in Indian law?   
2           Which law did BNS replace in Indian law?   
3                Is BNS applicable throughout India?   
4  What is the primary objective of BNS in Indian...   
5     Does BNS include both civil and criminal laws?   

                                              answer  
0            BNS stands for Bharatiya Nyaya Sanhita.  
1  BNS was introduced in 2023 and is set t

#### *Important*!

> ‼️Please choose the apt editing length here! (Number of prompts you want to choose inorder to edit the model)‼️



In [9]:
data_length=len(editing_prompts) #720 is the max
editing_length=7 # choose the apt editing length of the dataset prompts

### Locality prompts

In [10]:
from datasets import load_dataset

locality_p_ds = load_dataset("sentence-transformers/natural-questions",split=f"train[:{editing_length}]")
locality_prompts=locality_p_ds["query"]
print(len(locality_prompts))

README.md:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/44.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100231 [00:00<?, ? examples/s]

7


## locality data to check the locality metroc

In [11]:
locality_inputs = {}


In [12]:
from datasets import load_dataset
#len(editing_prompts) above is 720
gk_data =load_dataset("MuskumPillerum/General-Knowledge", split=f"train[:{editing_length}]")
print(len(gk_data["Question"]))
print(gk_data["Question"][:5])
print(gk_data["Answer"][:5])

locality_inputs["general_knowledge"]={
    "prompt": gk_data["Question"],
    "ground_truth": gk_data["Answer"]
}
print(len(locality_inputs["general_knowledge"]["prompt"]))
print(len(locality_inputs["general_knowledge"]["ground_truth"]))
assert len(locality_inputs["general_knowledge"]["prompt"]) == editing_length , "Choose the apt length for the dataset in the split"


README.md:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

output.json:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/37635 [00:00<?, ? examples/s]

7
['What is Artificial Intelligence?', 'What are the two main categories of Artificial Intelligence?', 'What is Machine Learning?', 'What is Deep Learning?', 'What is Natural Language Processing?']
['Artificial Intelligence refers to the development of computer systems that can perform tasks that would typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.\\n', 'The two main categories of Artificial Intelligence are Narrow AI and General AI. Narrow AI focuses on a specific task, while General AI aims to have human-like intelligence and the ability to learn and perform various tasks.\\n', 'Machine Learning is a subset of Artificial Intelligence that focuses on the development of algorithms that can learn from and make predictions or decisions on data.\\n', 'Deep Learning is a subset of Machine Learning that uses deep neural networks with many layers to learn from and make predictions on complex data.\\n', 'Natural 

In [13]:
from datasets import load_dataset
law_ds = load_dataset("viber1/indian-law-dataset", split=f"train[:{editing_length}]")
print(len(law_ds["Instruction"]))
print(law_ds["Instruction"][:5])
print(law_ds["Response"][:5])

locality_inputs["law_ds"]={
    "prompt": law_ds["Instruction"],
    "ground_truth": law_ds["Response"]
}
print(len(locality_inputs["law_ds"]["prompt"]))
print(len(locality_inputs["law_ds"]["ground_truth"]))
assert len(locality_inputs["law_ds"]["prompt"]) == editing_length , "Choose the apt length for the dataset in the split"

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24607 [00:00<?, ? examples/s]

7
['What is the difference between a petition and a plaint in Indian law?', 'When should a writ petition be filed in India?', 'What is the procedure for filing a plaint in a civil case in India?', 'What are the common reliefs sought through a public interest litigation (PIL) petition in India?', 'Can a plaint be amended after it has been filed in a civil case in India?']
["A petition is a formal request submitted to a courttribunalor authority to seek a specific remedy or relief. It is commonly used for various purposessuch as filing a writ petition in the High Court or submitting a petition for divorce. On the other handa plaint is a formal written statement of a plaintiff's claim in a civil lawsuit. The key difference is that a petition is more versatile and can be used for various legal matterswhile a plaint is specific to civil cases.", 'A writ petition in India should be filed when a fundamental right of an individual is violatedor when there is an issue related to the violation o

In [14]:
locality_inputs.keys()

dict_keys(['general_knowledge', 'law_ds'])

In [15]:
editing_prompts=editing_prompts[:editing_length]
editing_prompts=["Answer briefly w.r.t the Indian law: " + prompt for prompt in editing_prompts]
editing_targets=editing_targets[:editing_length]
rephrased_prompts=rephrased_prompts[:editing_length]
rephrased_prompts=["Answer briefly w.r.t the Indian law: " + prompt for prompt in rephrased_prompts]
assert len(editing_prompts)==len(rephrased_prompts),"Lengths of datasets should be same"

## Editing the model

1. Import the hparams file defined previously
2. Edit the model using the hparams
3. editing_prompts for editing the model
4. Rephrased prompts to check for reprhased_acc or generality metric
5. locality_inputs to check for locality metric

In [16]:
hparams = WISEHyperParams.from_hparams('./hparams/WISE/gpt2-medium.yaml') # loading the hparams file for gpt-2 medium

#hparams = WISEHyperParams.from_hparams('./hparams/WISE/gpt2-large.yaml') # !! if using gpt2-large
#hparams = WISEHyperParams.from_hparams('./hparams/WISE/gpt2-xl.yaml') # !! if using gpt2-xl

#Editing the model
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=editing_prompts,
    target_new=editing_targets,
    rephrase_prompts=rephrased_prompts,
    loc_prompts=locality_prompts,
    locality_inputs=locality_inputs,
    sequential_edit=True
)



We are creating the logger files


  0%|          | 0/7 [00:00<?, ?it/s]

New weights successfully inserted into transformer.h[8].mlp.c_fc.weight
Executing WISE algorithm for the update: 
[Answer briefly w.r.t the Indian law: What does BNS stand for in the context of Indian law?] -> [BNS stands for Bharatiya Nyaya Sanhita.]


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


loss 36.933 = 6.933 + 30.0
loss 32.181 = 4.813 + 27.368
loss 23.854 = 4.407 + 19.446
loss 15.541 = 3.598 + 11.943
loss 11.396 = 3.87 + 7.526
loss 9.348 = 3.122 + 6.226
loss 11.211 = 3.311 + 7.9
loss 5.827 = 2.867 + 2.96
loss 5.974 = 2.212 + 3.762
loss 6.403 = 2.405 + 3.997
loss 4.098 = 2.008 + 2.09
loss 5.146 = 2.066 + 3.08
loss 3.115 = 1.542 + 1.573
loss 3.536 = 1.549 + 1.987
loss 3.199 = 1.151 + 2.049
loss 3.085 = 1.175 + 1.91
loss 2.725 = 0.879 + 1.846
loss 2.382 = 0.932 + 1.45
loss 2.672 = 0.716 + 1.956
loss 2.082 = 0.802 + 1.28
loss 1.984 = 0.67 + 1.315
loss 2.132 = 0.677 + 1.455
loss 1.767 = 0.544 + 1.223
loss 1.511 = 0.547 + 0.964
loss 0.383 = 0.383 + 0.0
loss 0.34 = 0.34 + 0.0
loss 0.375 = 0.311 + 0.063
loss 1.83 = 0.419 + 1.411
loss 0.346 = 0.346 + 0.0
loss 0.307 = 0.307 + 0.0
loss 0.285 = 0.285 + 0.0
loss 0.268 = 0.268 + 0.0
loss 0.254 = 0.254 + 0.0
loss 0.243 = 0.243 + 0.0
loss 0.233 = 0.233 + 0.0
loss 0.224 = 0.224 + 0.0
loss 0.216 = 0.216 + 0.0
loss 0.209 = 0.209 + 0.0
los

...

 86%|████████▌ | 6/7 [01:12<00:12, 12.34s/it]

loss 0.115 = 0.115 + 0.0
Executing WISE algorithm for the update: 
[Answer briefly w.r.t the Indian law: Who enforces BNS in the context of Indian law?] -> [BNS is enforced by the police and judiciary system of India.]
loss 5.281 = 4.618 + 0.663
loss 3.633 = 2.788 + 0.845
loss 3.416 = 2.021 + 1.395
loss 1.787 = 1.32 + 0.467
loss 0.956 = 0.949 + 0.007
loss 1.344 = 0.765 + 0.579
loss 2.224 = 0.842 + 1.382
loss 0.943 = 0.507 + 0.436
loss 0.396 = 0.396 + 0.0
loss 0.319 = 0.319 + 0.0
loss 0.277 = 0.277 + 0.0
loss 0.25 = 0.25 + 0.0
loss 0.232 = 0.232 + 0.0
loss 0.218 = 0.218 + 0.0
loss 0.207 = 0.207 + 0.0
loss 0.662 = 0.224 + 0.438
loss 1.274 = 0.28 + 0.994
loss 0.461 = 0.217 + 0.244
loss 0.203 = 0.203 + 0.0
loss 0.192 = 0.192 + 0.0
loss 0.183 = 0.183 + 0.0
loss 0.176 = 0.176 + 0.0
loss 0.17 = 0.17 + 0.0
loss 0.164 = 0.164 + 0.0
loss 0.159 = 0.159 + 0.0
loss 0.155 = 0.155 + 0.0
loss 0.15 = 0.15 + 0.0
loss 0.146 = 0.146 + 0.0
loss 0.143 = 0.143 + 0.0
loss 0.139 = 0.139 + 0.0
loss 0.136 = 0.13

100%|██████████| 7/7 [01:23<00:00, 11.96s/it]

loss 0.111 = 0.111 + 0.0


Metrics Summary:  {'pre': {'rewrite_acc': 0.490092260680496, 'rephrase_acc': 0.44014221073044607}, 'post': {'rewrite_acc': 0.7057778822484705, 'rephrase_acc': 0.6088636853342735, 'locality': {'law_ds_acc': 0.8592794997233061, 'general_knowledge_acc': 0.8509364152097296}}}


* ‼️‼️ Editing Metrics output above! ‼️‼️ *
**If not visible, please scroll down to the bottom of the editing logs**

# Compiling the pre and the post-edited outputs in a csv file

In [17]:
results = []
gpt2_model = gpt2_model
gpt2_tokenizer.pad_token_id = gpt2_tokenizer.eos_token_id

# Process prompts in batches
batch_size = 10  # Adjust as needed based on your GPU memory
for i in range(0, len(editing_prompts), batch_size):
    batch_prompts = editing_prompts[i:i + batch_size]

    # Clear cache before each batch
    torch.cuda.empty_cache()

    batch = gpt2_tokenizer(batch_prompts, return_tensors="pt", padding=True).to(device)

    pre_edit_outputs = gpt2_model.generate(
        input_ids=batch['input_ids'].to(gpt2_model.device),
        attention_mask=batch['attention_mask'].to(gpt2_model.device),
        max_new_tokens=45, #Please increase the max token length as required for longer, complete outputs
    )

    post_edit_outputs = edited_model.generate(
        input_ids=batch['input_ids'].to(edited_model.device),
        attention_mask=batch['attention_mask'].to(edited_model.device),
        max_new_tokens=45, # Please increase the max token length as required for longer, complete outputs
    )

    max_length = batch['input_ids'].shape[-1]
    for j in range(len(batch_prompts)):
        print(editing_prompts[i+j])
        print(editing_targets[i+j])
        results.append([editing_prompts[i+j], editing_targets[i+j], gpt2_tokenizer.decode(pre_edit_outputs[j][max_length:], skip_special_tokens=True, num_beams=5, do_sample=True), gpt2_tokenizer.decode(post_edit_outputs[j][max_length:], skip_special_tokens=True, num_beams=5, do_sample=True)])
        print(i+j) #print total count

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Answer briefly w.r.t the Indian law: What does BNS stand for in the context of Indian law?
BNS stands for Bharatiya Nyaya Sanhita.
0
Answer briefly w.r.t the Indian law: When was BNS introduced in the context of Indian law?
BNS was introduced in 2023 and is set to replace IPC in 2024.
1
Answer briefly w.r.t the Indian law: What law did BNS replace in the context of Indian law?
BNS replaces the Indian Penal Code (IPC).
2
Answer briefly w.r.t the Indian law: Is BNS applicable across all of India in the context of Indian law?
Yes, BNS is applicable across all of India.
3
Answer briefly w.r.t the Indian law: What is the main purpose of BNS in the context of Indian law?
The main purpose of BNS is to modernize and streamline India's criminal laws.
4
Answer briefly w.r.t the Indian law: Does BNS cover both civil and criminal laws in the context of Indian law?
No, BNS only covers criminal laws. Civil laws are governed separately.
5
Answer briefly w.r.t the Indian law: Who enforces BNS in the c

In [18]:
results

[['Answer briefly w.r.t the Indian law: What does BNS stand for in the context of Indian law?',
  'BNS stands for Bharatiya Nyaya Sanhita.',
  'The answer is that it stands for the British Nationality Act of 1947. The British Nationality Act of 1947 was passed by Parliament in 1947 and was the first law to recognise the British Indian nationality. The British Nationality Act',
  'The BNS stands for the law of the State of India. It is the law of the State of India. It is the law of the State of India. It is the law of the State of India. It is'],
 ['Answer briefly w.r.t the Indian law: When was BNS introduced in the context of Indian law?',
  'BNS was introduced in 2023 and is set to replace IPC in 2024.',
  'The answer is that it was introduced in the context of Indian law. The Indian law was not introduced in the context of Indian law. The Indian law was introduced in the context of Indian law. The Indian law was introduced in',
  'The BNS was introduced in the context of Indian law 

In [19]:
%cd ..

/content


In [20]:
import pandas as pd

df=pd.DataFrame(results,columns=['Prompt', 'Ground Truth', 'Pre-Edit Output', 'Post-Edit Output'])
df.to_csv("wise_editing_outputs.csv",index=False)

### The dataset is output by the name of "wise_editing_outputs.csv"